In [1]:
import binaryninja as binja
import glob
from collections import defaultdict
import mongowrapper.MongoWrapper as mdb
import os
import pdb
import numpy as np
import pickle as p
import factorgraph as fg

In [2]:
def build_string_table(bv, db = None):
    addr = []
    string_dict = defaultdict(list)
    for key in bv.sections.keys():
        if 'rodata' in key:
            start = bv.sections[key].start
            end = bv.sections[key].end
            for string in bv.get_strings(start, end):
                addr.append(string.start)
    if addr == []:
        for string in bv.get_strings():
            addr.append(string.start)
    
    for func in bv.functions:
        
        string_list = []
        for block in func.low_level_il.basic_blocks:
            for instr in block:
                for oper in instr.operands:
                    if isinstance(oper, binja.LowLevelILInstruction):
                        if oper.operation == binja.LowLevelILOperation.LLIL_CONST_PTR or oper.operation == binja.LowLevelILOperation.LLIL_CONST:
                            if int(str(oper), 16) in addr:
                                string_list.append(bv.get_string_at(int(str(oper), 16)).value)
                
        if len(string_list) > 0:
            string_list_strip = list(set(string_list))
            my_dict = {"name": func.name, "strings": string_list_strip}
            string_dict[func.name] = string_list_strip
            if db is not None:
                db.save(my_dict)
    return string_dict


In [90]:
#string_table = build_string_table(binaryview)
p.dump(string_table, open('string_dict.p','wb'))

In [54]:
binaryview.get_string_at(9871340)

<StringType.AsciiString: 0x969fc7, len 0xb9>

In [3]:
# libraryName = 'libcrypto'
# db = mdb('oss', libraryName + '_stringtable')
# global path_lib
#path_lib = '/home/yijiufly/Downloads/codesearch/data/openssl/'
# path_lib = '/home/yijiufly/Downloads/codesearch/data/zlib/zlib-O2/'
# for folder in os.listdir(path_lib):
#     folder = 'zlib_1_2_11'
#     print folder
#     bv = binja.BinaryViewType.get_view_of_file(path_lib + folder + '/libz.so')
#     build_string_table(bv, db)
#     break
libraryName = 'library'
mongodb = mdb('oss', libraryName + '_stringtable')

In [42]:
def loadFiles(PATH, ext=None):  # use .ida or .emb for ida file and embedding file
    filenames = []
    filenames = [PATH + f for f in os.listdir(PATH) if f.endswith(ext)]
    return filenames
libraryName = 'library'
mongodb = mdb('oss', libraryName + '_stringtable')
global path_lib
path_lib = '/home/yijiufly/Downloads/codesearch/data/openssl/openssl-1.0.1d/objfiles-libcrypto/'
#path_lib = '/home/yijiufly/Downloads/codesearch/data/zlib/zlib-O2/'
stringfiles = loadFiles(path_lib, ext='.str')
stringfiles.extend(loadFiles('/home/yijiufly/Downloads/codesearch/data/openssl/openssl-1.0.1d/objfiles-libssl/', ext='.str'))
stringfiles.extend(loadFiles('/home/yijiufly/Downloads/codesearch/data/zlib/zlib-O2/zlib-1.2.11/objfiles/', ext='.str'))
globalstring_dict = defaultdict(list)
for s in stringfiles:
    string_dict = p.load(open(s, 'rb'))
    for key in string_dict.keys():
        my_dict = {"name": key, "strings": string_dict[key]}
        globalstring_dict[key] = string_dict[key]
        print my_dict
        mongodb.save(my_dict)


{'name': u'mdc2_body', 'strings': [u'WAVAUATUSH']}
{'name': u'MDC2_Final', 'strings': [u'WAVAUATUSH']}
{'name': u'MDC2_Init', 'strings': [u'WAVAUATUSH']}
{'name': u'MDC2_Update', 'strings': [u'WAVAUATUSH']}
{'name': u'OPENSSL_config', 'strings': [u'Auto configuration failed\n']}
{'name': u'DES_ofb64_encrypt', 'strings': [u'WAVI']}
{'name': u'CRYPTO_cfb128_1_encrypt', 'strings': [u'WAVI']}
{'name': u'a2i_ASN1_INTEGER', 'strings': [u'f_int.c']}
{'name': u'hash_block', 'strings': [u'WAVI']}
{'name': u'finish_hash', 'strings': [u'WAVI']}
{'name': u'hash_step', 'strings': [u'WAVI']}
{'name': u'init_gost_hash_ctx', 'strings': [u'WAVI']}
{'name': u'start_hash', 'strings': [u'WAVI']}
{'name': u'int_ctrl_helper.isra.1', 'strings': [u'eng_ctrl.c']}
{'name': u'ENGINE_ctrl', 'strings': [u'eng_ctrl.c']}
{'name': u'ENGINE_ctrl_cmd', 'strings': [u'eng_ctrl.c']}
{'name': u'ENGINE_cmd_is_executable', 'strings': [u'eng_ctrl.c']}
{'name': u'ENGINE_ctrl_cmd_string', 'strings': [u'eng_ctrl.c']}
{'name': u'

{'name': u'DHparams_print_fp', 'strings': [u'dh_prn.c']}
{'name': u'c2i_ASN1_BIT_STRING', 'strings': [u'a_bitstr.c']}
{'name': u'ASN1_BIT_STRING_set_bit', 'strings': [u'a_bitstr.c']}
{'name': u'pkey_cmac_ctrl_str', 'strings': [u'hexkey', u'cipher']}
{'name': u'SRP_create_verifier', 'strings': [u'srp_vfy.c']}
{'name': u'SRP_VBASE_new', 'strings': [u'srp_vfy.c']}
{'name': u'SRP_VBASE_init', 'strings': [u'srp_vfy.c']}
{'name': u'SRP_gN_new_init', 'strings': [u'srp_vfy.c']}
{'name': u'SRP_VBASE_get_by_user', 'strings': [u'srp_vfy.c']}
{'name': u'EVP_SealInit', 'strings': [u'WAVI']}
{'name': u'x509_name_ex_d2i', 'strings': [u'x_name.c']}
{'name': u'x509_name_ex_i2d', 'strings': [u'x_name.c']}
{'name': u'x509_name_ex_new', 'strings': [u'x_name.c']}
{'name': u'x509_name_canon.part.0', 'strings': [u'x_name.c']}
{'name': u'BF_options', 'strings': [u'blowfish(idx)']}
{'name': u'X509_NAME_oneline', 'strings': [u'NO X509_NAME', u'x509_obj.c']}
{'name': u'd2i_ASN1_type_bytes', 'strings': [u'a_bytes

{'name': u'EVP_DigestSignInit', 'strings': [u'm_sigver.c']}
{'name': u'EVP_DigestVerifyInit', 'strings': [u'm_sigver.c']}
{'name': u'OBJ_txt2nid', 'strings': [u'obj_dat.c']}
{'name': u'OBJ_txt2obj', 'strings': [u'obj_dat.c']}
{'name': u'OBJ_nid2sn', 'strings': [u'obj_dat.c']}
{'name': u'OBJ_obj2txt', 'strings': [u'obj_dat.c', u'.%lu']}
{'name': u'OBJ_create', 'strings': [u'obj_dat.c']}
{'name': u'OBJ_nid2ln', 'strings': [u'obj_dat.c']}
{'name': u'OBJ_add_object', 'strings': [u'obj_dat.c']}
{'name': u'OBJ_nid2obj', 'strings': [u'obj_dat.c']}
{'name': u'd2i_EC_PUBKEY', 'strings': [u'x_pubkey.c']}
{'name': u'i2d_DSA_PUBKEY', 'strings': [u'x_pubkey.c']}
{'name': u'X509_PUBKEY_get', 'strings': [u'x_pubkey.c']}
{'name': u'i2d_PUBKEY', 'strings': [u'x_pubkey.c']}
{'name': u'X509_PUBKEY_set', 'strings': [u'x_pubkey.c']}
{'name': u'd2i_PUBKEY', 'strings': [u'x_pubkey.c']}
{'name': u'd2i_DSA_PUBKEY', 'strings': [u'x_pubkey.c']}
{'name': u'i2d_RSA_PUBKEY', 'strings': [u'x_pubkey.c']}
{'name': u'i

{'name': u'DSO_set_filename', 'strings': [u'dso_lib.c']}
{'name': u'DSO_convert_filename', 'strings': [u'dso_lib.c']}
{'name': u'DSO_pathbyaddr', 'strings': [u'dso_lib.c']}
{'name': u'DSO_global_lookup', 'strings': [u'dso_lib.c']}
{'name': u'DSO_load', 'strings': [u'dso_lib.c']}
{'name': u'DSO_bind_var', 'strings': [u'dso_lib.c']}
{'name': u'DSO_free', 'strings': [u'dso_lib.c']}
{'name': u'DSO_new', 'strings': [u'dso_lib.c']}
{'name': u'DSO_ctrl', 'strings': [u'dso_lib.c']}
{'name': u'DSO_get_filename', 'strings': [u'dso_lib.c']}
{'name': u'DSO_set_name_converter', 'strings': [u'dso_lib.c']}
{'name': u'DSO_new_method', 'strings': [u'dso_lib.c']}
{'name': u'DSO_merge', 'strings': [u'dso_lib.c']}
{'name': u'DSO_up_ref', 'strings': [u'dso_lib.c']}
{'name': u'X509_REQ_print_fp', 'strings': [u'critical', u'    Data:\n', u'%12sUnable to load Public Key\n', u'        Subject Public Key Info:\n', u': %s\n', u'%16s', u't_req.c', u'Certificate Request:\n', u'%12sa0:00\n', u'%8sRequested Extensio

{'name': u'dtls1_check_timeout_num', 'strings': [u'd1_lib.c']}
{'name': u'ssl_parse_clienthello_renegotiate_ext', 'strings': [u't1_reneg.c']}
{'name': u'ssl_add_serverhello_renegotiate_ext', 'strings': [u't1_reneg.c']}
{'name': u'ssl_parse_serverhello_renegotiate_ext', 'strings': [u'!expected_len || s->s3->previous_client_finished_len', u't1_reneg.c', u'!expected_len || s->s3->previous_server_finished_len']}
{'name': u'ssl_add_clienthello_renegotiate_ext', 'strings': [u't1_reneg.c']}
{'name': u'ssl2_enc_init', 'strings': [u's->s2->key_material_length <= sizeof s->s2->key_material', u's2_enc.c', u'c->iv_len <= (int)sizeof(s->session->key_arg)']}
{'name': u'ssl2_set_certificate', 'strings': [u's2_clnt.c']}
{'name': u'ssl2_connect', 'strings': [u's2_clnt.c']}
{'name': u'tls1_mac', 'strings': [u't > 0', u't >= 0', u't1_enc.c']}
{'name': u'tls1_enc', 'strings': [u'%s:%d: rec->data != rec->input\n', u't1_enc.c', u'n >= 0']}
{'name': u'tls1_P_hash.constprop.2', 'strings': [u'chunk >= 0', u't1

In [60]:
for key in string_table.keys():
    if key in globalstring_dict:
        if set(globalstring_dict[key]) != set(string_table[key]):
            print key, globalstring_dict[key], string_table[key]

dynamic_ctrl [u'eng_dyn.c'] ['v_check', 'eng_dyn.c', 'bind_engine']
SMIME_read_ASN1 [u'application/x-pkcs7-signature', u'application/x-pkcs7-mime', u'asn_mime.c', u'multipart/signed', u'application/pkcs7-mime', u'application/pkcs7-signature', u'type: '] ['application/x-pkcs7-signature', 'application/pkcs7-signature', 'asn_mime.c', 'multipart/signed', 'type: ', 'application/x-pkcs7-mime', 'application/pkcs7-mime', 'content-type']
HMAC_Init [u'len>=0 && len<=(int)sizeof(ctx->key)', u'j <= (int)sizeof(ctx->key)', u'hmac.c', u'6666666666666666\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'] ['6666666666666666jjjjjjjjjjjjjjjjd1_both.c', 'hmac.c', '\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\bn_lib.c', 'len>=0 && len<=(int)sizeof(ctx->key)', u'6666666666666666\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\', u'j <= (int)sizeof(ctx->key)']
rc4_hmac_md5_ctrl [u'6666666666666666jjjjjjjjjjjjjjjj'] ['6666666666666666jjjjjjjjjjjjjjjjd1_both.c', u'6666666666666666jjjjjjjjjjjjjjjj']
EVP_CipherInit_ex [u'ctx->cipher->block_size == 1 || ctx-

In [6]:
string_table['ENGINE_get_last']

['eng_list.c']

In [4]:
global LBP_MAX_ITERS
global THRES
LBP_MAX_ITERS =40
THRES = 0.99
knn = p.load(open('/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/test_kNN_1021_2gram.p', 'r'))
binaryview = binja.BinaryViewType.get_view_of_file('/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/nginx-{openssl-1.0.1d}{zlib-1.2.11}')
string_table = build_string_table(binaryview)

In [5]:
def BP_with_strings(queryPath_1gram, queryPath_2gram, string_dict, db, positiveinfluence, negativeinfluence):
    global LBP_MAX_ITERS
    global THRES
    threshold=THRES
    g = fg.Graph()
    global_node_dict = dict()
    global_edge_dict = dict()
    global_sim_dict = defaultdict(float)
    rvlabels = defaultdict(list)
    if type(queryPath_2gram) == type(''):
        query_2gram = p.load(open(queryPath_2gram, 'rb'))
        query_1gram = p.load(open(queryPath_1gram, 'rb'))
    else:
        query_2gram = queryPath_2gram
        query_1gram = queryPath_1gram
    hasDistance = False
    if type(query_2gram[0][0][0]) == type((1,2)):
        hasDistance = True

    global_node_dict, global_prior_dict = get_node_prior_belief(query_1gram)
    for i in query_2gram:
        if i[2] > threshold:
            if not hasDistance:
                test_src = i[0][0]
                test_des = i[0][1]
            else:
                test_src = i[0][0][0]
                test_des = i[0][0][1]
            if test_src == test_des:
                continue
#             # initialize the dictionary
            if not global_node_dict.has_key(test_src):
                global_node_dict[test_src] = []
            if not global_node_dict.has_key(test_des):
                global_node_dict[test_des] = []
            if not global_edge_dict.has_key((test_src, test_des)):
                global_edge_dict[(test_src, test_des)] = []

            funcList = i[1]
            for predicted_func in funcList:
                if hasDistance:
                    querydistance = i[0][1]
                    resultdistance = predicted_func[3]
                    if querydistance != resultdistance:
                        continue

                src_funcname = predicted_func[2][0]
                des_funcname = predicted_func[2][1]
                libraryname = predicted_func[0]

                # find the src function if it's already in the dict,
                # otherwise, add it to the dict
                if (src_funcname, libraryname) not in global_node_dict[test_src]:
                    continue
                # do the same thing to des function
                if (des_funcname, libraryname) not in global_node_dict[test_des]:
                    continue
                
                # add this edge to the edge dictionary
                if (src_funcname, des_funcname, libraryname) not in global_edge_dict[(test_src, test_des)]:
                    global_edge_dict[(test_src, test_des)].append((src_funcname, des_funcname, libraryname))
                    if global_sim_dict[(src_funcname, des_funcname, libraryname)] < i[2]:
                        global_sim_dict[(src_funcname, des_funcname, libraryname)] = i[2]
                    
    string_count = defaultdict(int)
    # add functions as random variables and add factors between function and string nodes
    for key in global_node_dict:
        rvlabels[test_src].append('None')
        length = len(global_node_dict[key])
        if length > 0:
            #rv_func = g.rv(key, length + 1, prior = global_prior_dict[key])
            rv_func = g.rv(key, length + 1)
        else:
            rv_func = g.rv(key, length + 1) # None label is the last item
        for idx, (prediction, library) in enumerate(global_node_dict[key]):
            query = db.load({"name":prediction})
            if not query:
                string_list = []
            else:
                string_list = query['strings']
            string_shared = set(string_dict[key]) & set(string_list)
            string_differ = set(string_dict[key]) | set(string_list) - string_shared
#             string_shared = set()
#             string_differ = set()
            #string_differ = set(string_list) - set(string_dict[key])
            for string in string_shared:
                count = string_count[string]
                string_count[string] += 1
                rv_string = g.rv('str_' + string + str(count), 1)
                fillvalue = 1.0
                pot=np.full((length + 1, 1), fillvalue)
                pot[idx, 0] = positiveinfluence * fillvalue
                factor = g.factor([rv_func, rv_string], potential=pot, name = '')
                
            for string in string_differ:
                count = string_count[string]
                string_count[string] += 1
                rv_string = g.rv('str_' + string + str(count), 1)    
                fillvalue = 1.0
                pot=np.full((length + 1, 1), positiveinfluence * fillvalue)
                pot[idx, 0] = fillvalue
                factor = g.factor([rv_func, rv_string], potential=pot, name = '')
        global_node_dict[key].append(('None', 'None'))
    p.dump(global_node_dict, open('global_node_dict','w'))


    # add factors
    for (test_src, test_des) in global_edge_dict.keys():
        src_rv = g.get_rvs()[test_src]
        des_rv = g.get_rvs()[test_des]
        src_list = global_node_dict[test_src]
        des_list = global_node_dict[test_des]
        # set the probability for src_rv and des_rv
        # first find if there is an existing probability, if so, add to it
        # otherwise, initialize a new probability matrix
        factor1 = None
        probability1 = None

        if test_src != test_des:
            pot=np.full((len(src_list), len(des_list)), 0.1)
            #pot[-1, -1] = 0.1
            factor1 = g.factor([src_rv, des_rv], potential=pot, name = '')
            probability1 = factor1.get_potential()

        # if there is an matching edge
        if len(global_edge_dict[(test_src, test_des)]) > 0:
            for (src_funcname, des_funcname, libraryname) in global_edge_dict[(test_src, test_des)]:
                sim = global_sim_dict[(src_funcname, des_funcname, libraryname)]
                src_ind = src_list.index((src_funcname, libraryname))
                des_ind = des_list.index((des_funcname, libraryname))
                if src_rv == factor1.get_rvs()[0]:
                    #probability1[src_ind, des_ind] = 2**(sim + 2) - 7
                    probability1[src_ind, des_ind] = positiveinfluence * 0.1
                else:
                    #probability1[des_ind, src_ind] = 2**(sim + 2) - 7
                    probability1[des_ind, src_ind] = positiveinfluence * 0.1
        else:
            if len(src_list) > 0 and len(des_list) > 0:
                probability1[:, -1] = negativeinfluence * 0.1
                probability1[-1, :] = negativeinfluence * 0.1
                
        factor1.set_potential(probability1)
        #factor1.name += src_funcname + '->' + des_funcname + '\n'

    iters, converged = g.lbp(global_node_dict, normalize=True, max_iters=LBP_MAX_ITERS, progress=False)
    g.global_dict = global_node_dict
    print 'LBP ran for %d iterations. Converged = %r' % (iters, converged)
#     results = g.get_func_prediction(normalize=True)
#     p.dump(results, open('prediction','w'))
#     results = g.accuracy_of_functions(normalize=True)
    return g

In [12]:
global THRES
THRES = 0.95
global LBP_MAX_ITERS
LBP_MAX_ITERS =40
knn_all = p.load(open('/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/test_kNN_1030_1gram.p'))
g = BP_with_strings(knn_all, knn, string_table, mongodb, 5, 5)

factorgraph/factorgraph.py:99: RuntimeWarning: invalid value encountered in divide
  c = a / b


LBP ran for 25 iterations. Converged = 0.9919829401284889


In [7]:
# input: [(func, sim), (func, sim)...]
def getRank(prediction):
    predict = sorted(prediction, key=lambda x: x[1])
    rank = 1
    lastsim = -1
    predicts = []
    ranks = []
    for (func, sim) in predict:
        if np.isclose(sim, lastsim):
            predicts.append(func)
            ranks.append(rank)
        else:
            predicts.append(func)
            ranks.append(rank)
            rank += 1
        lastsim = sim
    sm = softmax(range(ranks[-1]+1))
    for i in range(len(ranks)):
        ranks[i] = sm[ranks[i]]
    ranks.append(sm[0])
    return predicts, ranks

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

def get_node_prior_belief(knn_all):
    prediction = defaultdict(set)
    sim_dict = defaultdict(float)
    for [(name, distance), predict, sim] in knn_all:
        if sim > 0.95:
            for [lib, version, func] in predict:
                prediction[name].add((func, lib))
                if sim_dict[(name, func, lib)] < sim:
                    sim_dict[(name, func, lib)] = sim
                    
    prediction_with_sim = defaultdict(list)
    
    for name in prediction.keys():
        for (func, lib) in prediction[name]:
            prediction_with_sim[name].append(((func, lib), sim_dict[(name, func, lib)]))
            
    global_dict = defaultdict(list)
    global_beliefs = defaultdict(list)
    for func in prediction_with_sim.keys():
        predicts, beliefs = getRank(prediction_with_sim[func])
        global_dict[func] = predicts
        global_beliefs[func] = beliefs
        ""
    return global_dict, global_beliefs

#get_node_prior_belief(knn_all)

In [8]:
def accuracy_of_functions(g, librarynames, rvs=None, normalize=True):
        # Extract
        global_dict = g.global_dict
        tuples = g.rv_marginals(rvs, normalize)
        prediction = dict()
        count = 0
        tn = 0
        fn = 0
        wp = 0
        fp = 0
        prior = defaultdict(list)
        # Display
        for rv, marg in tuples:
            rv_label = str(rv).split(' ')[0]
            if rv_label[:4] == 'str_':
                continue
            largest = -1
            for label in marg:
                if label > largest:
                    largest = label
            names = global_dict[rv_label]

            # if np.isclose(largest, 1.0/len(names)):
            #     continue
            for i, label in enumerate(marg):
                prior[rv_label].append((names[i], label))
                if np.isclose(label, largest):
                    #pdb.set_trace()
                    if rv_label in prediction:
                        prediction[rv_label].append(names[i])
                    else:
                        prediction[rv_label]=[names[i]]

        keylist = prior.keys()
        # for key in keylist:
        #     print key
        #     print prior[key]
        #     print
        keylist = prediction.keys()
        correct_set = set()
        for key in keylist:
            funclist = prediction[key]
            label=set()
            #print key
            for (func, lib) in funclist:
                if len(func.split('{')) > 1:
                    predicted_label = func.split('{')[1].split('}')[0]
                else:
                    predicted_label = func
                label.add(predicted_label)
            if 'None' in label and len(label)==1:
                if key in librarynames:
                    fn += 1
                else:
                    tn += 1
            elif key in label:
                count += 1
                correct_set.add(key)
                wp += len(label)-1
            else:
                if key in librarynames:
                    wp += len(label)
                else:
                    fp += len(label)

        print "correct prediction, true negative, false negative, wrong prediction, false positives, total prediction, precision:"
        print count, tn, fn, wp, fp, len(keylist), count*1.0/(len(keylist)-tn-fn), count*1.0/2982
        return count*1.0/(len(keylist)-tn-fn), count*1.0/2982

In [25]:
precision, recall = accuracy_of_functions(g, librarynames)

correct prediction, true negative, false negative, wrong prediction, false positives, total prediction, precision:
2380 1378 371 1648 58 4324 0.92427184466 0.798122065728


In [9]:
filted = 1
def loadOneBinary(funcnamepath, embFile, filted_size=0):
    names = p.load(open(funcnamepath, 'r'))
    data = p.load(open(embFile, 'r'))
    funcName2emb=dict()
    for i in range(len(data)):
        (name, size) = names[i]
        if size > filted_size:
            funcName2emb[name]=data[i]
    return funcName2emb
#dir_openssl = '/home/yijiufly/Downloads/codesearch/Gemini/testingdataset/database/'
dir_openssl = '/home/yijiufly/Downloads/codesearch/data/openssl/'
dir_zlib = '/home/yijiufly/Downloads/codesearch/data/zlib/zlib-O2/'
#folder = '101d/'
folder = 'openssl-1.0.1d/'
embfile = 'libcrypto.so_newmodel.emb'
nam = 'libcrypto.so_newmodel_withsize.nam'
sslembfile = 'libssl.so_newmodel.emb'
sslnam = 'libssl.so_newmodel_withsize.nam'
func2emb_ssl = loadOneBinary(dir_openssl+folder+sslnam, dir_openssl+folder+sslembfile, filted)
#name_filted = p.load(open(dir_openssl+'libcrypto.so.ida_filted1.nam','r'))
func2emb_openssl = loadOneBinary(dir_openssl+folder+nam, dir_openssl+folder+embfile, filted)
func2emb_zlib = loadOneBinary(dir_zlib+'zlib-1.2.11/libz.so_newmodel_withsize.nam', dir_zlib+'zlib-1.2.11/libz.so_newmodel.emb', filted)
librarynames = func2emb_openssl.keys()
librarynames.extend(func2emb_ssl.keys())
librarynames.extend(func2emb_zlib.keys())
print len(librarynames)

3451


In [312]:
print len(g.global_dict.keys())
truth=[]
for key in g.global_dict:
    pre = [i[0] for i in g.global_dict[key]]
    if key in pre:
        truth.append(key)
print len(truth)

print set(truth)-correct_set, len(set(truth)-correct_set)

4142
1820
set(['name_cmp', 'X509V3_section_free', 'atalla_ctrl', 'dtls1_get_message_fragment', 'ENGINE_new', 'ENGINE_get_prev', 'PEM_ASN1_write', 'i2a_ASN1_OBJECT', 'X509_print_ex', 'ubsec_mod_exp_dh', 'surewarehk_rsa_priv_dec', 'RSA_padding_add_PKCS1_PSS', 'ecdsa_check', 'EC_KEY_insert_key_method_data', 'b64_write', 'SSL_CTX_use_certificate_chain_file', 'CRYPTO_add_lock', 'aep_finish', 'BIO_push', 'X509_EXTENSION_get_object', 'UI_dup_verify_string', 'EVP_PKEY_keygen', 'SSL_CTX_SRP_CTX_free', 'hwcrhk_load_privkey', 'dtls1_retransmit_message', 'ec_asn1_group2parameters.constprop.3', 'CRYPTO_pop_info', 'ASN1_item_d2i_fp', 'sk_dup', 'ENGINE_register_all_DH', 'DSO_load', 'BN_sub_word', 'DSA_new_method', 'BIO_vsnprintf', 'SSL_CTX_use_RSAPrivateKey_file', 'nuron_ctrl', 'CONF_get_number', 'i2r_pci', 'dtls1_do_write', 'RSA_verify_PKCS1_PSS_mgf1', 'ENGINE_init', 'EC_KEY_new', 'HMAC_Init_ex', 'str_copy', 'EC_KEY_new_by_curve_name', 'asn1_get_field_ptr', 'do_ssl3_write', 'surewarehk_ctrl', 'pkey_

In [292]:
print prediction['ngx_snprintf']

[('des_ede_cbc_cipher.constprop.3', 'libcrypto'), ('des_ede_cbc_cipher.constprop.4', 'libcrypto')]


In [293]:
print belief['ngx_snprintf']

[(('ssl3_output_cert_chain', 'libssl'), 0.00830847411380814), (('SSL_use_certificate', 'libssl'), 0.008308474113808145), (('SSL_use_PrivateKey', 'libssl'), 0.00830847411380814), (('RIPEMD160', 'libcrypto'), 0.11726759582467496), (('MD4', 'libcrypto'), 0.11470638604968199), (('MD5', 'libcrypto'), 0.11470638604968202), (('des_ede_cbc_cipher.constprop.3', 'libcrypto'), 0.21400985991435587), (('des_ede_cbc_cipher.constprop.4', 'libcrypto'), 0.21400985991435587), (('idea_cbc_cipher', 'libcrypto'), 0.11729748085384512), (('None', 'None'), 0.0830770090519797)]


[(('BN_div', 'libcrypto'), 0.10824380223553921),
 (('BN_generate_prime', 'libcrypto'), 0.020243849048146516),
 (('ssl3_read_bytes', 'libssl'), 0.7993934659763151),
 (('ASN1_item_free', 'libcrypto'), 0.0006476214819616707),
 (('ssl3_write_bytes', 'libssl'), 0.012596098828642273),
 (('do_ssl3_write', 'libssl'), 0.001768937875641718),
 (('parse_bags.constprop.1', 'libcrypto'), 0.001251114814716462),
 (('do_dtls1_write', 'libssl'), 0.0008966877847646794),
 (('asn1_parse2', 'libcrypto'), 6.036906078289807e-17),
 (('ssl_cipher_apply_rule.isra.1.constprop.3', 'libssl'),
  0.0010030327505468795),
 (('BN_generate_prime_ex', 'libcrypto'), 0.005721200369863713),
 (('X509_verify_cert', 'libcrypto'), 2.399350780754784e-05),
 (('ssl_cipher_get_evp', 'libssl'), 0.0006140690736818583),
 (('BN_mod_inverse', 'libcrypto'), 0.0009071887791236199),
 (('ssl_parse_clienthello_tlsext', 'libssl'), 0.03952635708815168),
 (('ssl2_accept', 'libssl'), 0.005480928190500681),
 (('dsa_builtin_paramgen2', 'libcrypto')

In [294]:
rvs = g.get_rvs()['ngx_snprintf']
for factor in rvs.get_factors():
    print factor
    print factor.get_outgoing_for(rvs)
    
    

KeyError: 'ngx_snprintf'

In [285]:
belief['ENGINE_get_next']

[(('ENGINE_get_next', 'libcrypto'), 0.9970496531404276),
 (('None', 'None'), 0.002950346859572456)]

In [ ]:
from simanneal import Annealer
import random
def F(beta, precision, recall):
    return (beta*beta + 1)*precision*recall / (beta*beta*precision + recall)
class FindBestParameters(Annealer):
    def __init__(self, state):
        #self.state = state
        super(FindBestParameters, self).__init__(state)
        
    def move(self):
        """Swaps two cities in the route."""
        a = random.uniform(0, 10)
        b = random.uniform(0, 10)
        self.state['positive'], self.state['negative'] = a, b
        
    def energy(self):
        """Calculates the length of the route."""
        print self.state['positive'], self.state['negative']
        g = BP_with_strings(knn_all, knn, string_table, mongodb, self.state['positive'], self.state['negative'])
        precision, recall = accuracy_of_functions(g, librarynames)
        return F(1, precision, recall)
    
init_state = {'positive':5.0, 'negative':5.0}
tsp = FindBestParameters(init_state)
tsp.set_schedule(tsp.auto(minutes=3))
state, e = tsp.anneal()
print state, e

5.0 5.0
LBP ran for 25 iterations. Converged = 0.9919829401284889


 Temperature        Energy    Accept   Improve     Elapsed   Remaining


correct prediction, true negative, false negative, wrong prediction, false positives, total prediction, precision:
1987 1391 816 1378 28 4324 0.938592347662 0.666331321261
0.752341932944 6.8886782089
LBP ran for 18 iterations. Converged = 0.9928197376234951
correct prediction, true negative, false negative, wrong prediction, false positives, total prediction, precision:
574 1305 1789 1942 134 4324 0.466666666667 0.192488262911
0.752341932944 6.8886782089
LBP ran for 18 iterations. Converged = 0.9928197376234951
correct prediction, true negative, false negative, wrong prediction, false positives, total prediction, precision:
574 1305 1789 1942 134 4324 0.466666666667 0.192488262911
2.58295755146 6.40875207381
LBP ran for 12 iterations. Converged = 0.9916590185175188
correct prediction, true negative, false negative, wrong prediction, false positives, total prediction, precision:
1903 1391 909 1324 28 4324 0.940217391304 0.638162307176
0.195445346362 5.37745026806
LBP ran for 19 iteratio

In [88]:
filted = 1
def loadOneBinary(funcnamepath, embFile, filted_size=0):
    names = p.load(open(funcnamepath, 'r'))
    data = p.load(open(embFile, 'r'))
    funcName2emb=dict()
    for i in range(len(data)):
        (name, size) = names[i]
        if size > filted_size:
            funcName2emb[name]=data[i]
    return funcName2emb
#dir_openssl = '/home/yijiufly/Downloads/codesearch/Gemini/testingdataset/database/'
dir_openssl = '/home/yijiufly/Downloads/codesearch/data/openssl/'
dir_zlib = '/home/yijiufly/Downloads/codesearch/data/zlib/zlib-O2/'
#folder = '101d/'
folder = 'openssl-1.0.1d/'
embfile = 'libcrypto.so_newmodel.emb'
nam = 'libcrypto.so_newmodel_withsize.nam'
sslembfile = 'libssl.so_newmodel.emb'
sslnam = 'libssl.so_newmodel_withsize.nam'
func2emb_ssl = loadOneBinary(dir_openssl+folder+sslnam, dir_openssl+folder+sslembfile, filted)
#name_filted = p.load(open(dir_openssl+'libcrypto.so.ida_filted1.nam','r'))
func2emb_openssl = loadOneBinary(dir_openssl+folder+nam, dir_openssl+folder+embfile, filted)
func2emb_zlib = loadOneBinary(dir_zlib+'zlib-1.2.11/libz.so_newmodel_withsize.nam', dir_zlib+'zlib-1.2.11/libz.so_newmodel.emb', filted)


In [85]:
dir_openssl = '/home/yijiufly/Downloads/codesearch/data/openssl/openssl-1.0.1d/emb_sharedlib/'
func2emb_ssl_old = loadOneBinary(dir_openssl+'libssl.so.ida_newmodel_withsize.nam', dir_openssl+'libssl.so.ida_newmodel.emb', filted)
#name_filted = p.load(open(dir_openssl+'libcrypto.so.ida_filted1.nam','r'))
func2emb_openssl_old = loadOneBinary(dir_openssl+'libcrypto.so.ida_newmodel_withsize.nam', dir_openssl+'libcrypto.so.ida_newmodel.emb', filted)

In [87]:
print len(set(func2emb_openssl.keys()) - set(func2emb_openssl_old.keys()))

399


In [91]:
global_node_dict = p.load(open('global_node_dict','r'))
predictcandidate = []
for key in global_node_dict.keys():
    if key in func2emb_openssl.keys() or key in func2emb_zlib.keys() or key in func2emb_ssl.keys():
        predict = [i[0] for i in global_node_dict[key]]
        if key in predict:
            predictcandidate.append(key)
print len(predictcandidate), len(global_node_dict.keys())

2203 3315


In [94]:
keys = set()
for i in knn:
    test_src = i[0][0][0]
    test_des = i[0][0][1]
    keys.add(test_src)
    keys.add(test_des)
predictcandidate=[]
for key in keys:
    if key in func2emb_openssl.keys() or key in func2emb_zlib.keys() or key in func2emb_ssl.keys():
        predictcandidate.append(key)
print len(predictcandidate)

2686


In [93]:
len(keys)

4143

[(u'_init', 3), (u'__ctype_toupper_loc', 1), (u'sub_4426a6', 2), (u'__strcat_chk', 1), (u'getenv', 1), (u'sigprocmask', 1), (u'raise', 1), (u'free', 1), (u'recv', 1), (u'pthread_create', 1), (u'pthread_sigmask', 1), (u'strcasecmp', 1), (u'localtime', 1), (u'__vfprintf_chk', 1), (u'abort', 1), (u'__errno_location', 1), (u'srandom', 1), (u'unlink', 1), (u'strncpy', 1), (u'strncmp', 1), (u'localtime_r', 1), (u'strcpy', 1), (u'mkdir', 1), (u'sendmsg', 1), (u'writev', 1), (u'puts', 1), (u'ferror', 1), (u'qsort', 1), (u'sigaction', 1), (u'fread', 1), (u'strtod', 1), (u'setsockopt', 1), (u'fcntl', 1), (u'write', 1), (u'getpid', 1), (u'pthread_cond_wait', 1), (u'crypt_r', 1), (u'fclose', 1), (u'__xstat64', 1), (u'opendir', 1), (u'shutdown', 1), (u'globfree64', 1), (u'rmdir', 1), (u'strlen', 1), (u'chdir', 1), (u'pthread_mutexattr_destroy', 1), (u'__stack_chk_fail', 1), (u'getuid', 1), (u'mmap', 1), (u'accept4', 1), (u'dup2', 1), (u'send', 1), (u'strchr', 1), (u'pthread_mutex_destroy', 1), (u'p

In [95]:
from binary import TestBinary
dir = '/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/'
dotfiles = loadFiles(dir, ext='_bn.dot')
namfiles = loadFiles(dir, ext='.ida_newmodel_withsize.nam')
embfiles = loadFiles(dir, ext='.ida_newmodel.emb')
dotfile = dotfiles[0]
namfile = namfiles[0]
embfile = embfiles[0]
dotPath = os.path.join(dir, dotfile)
namPath = os.path.join(dir, namfile)
namPathFull = os.path.join(dir, namfile[:-13]+'.nam')
embPath = os.path.join(dir, embfile)
testbin = TestBinary('nginx', dotPath, embPath, namPath, 1)
testbin.loadOneBinary(namPathFull, testbin.embFile)
connectednodes = set()
for src in testbin.callgraphEdges.keys():
    for (des, distance) in testbin.callgraphEdges[src]:
        connectednodes.add(src)
        connectednodes.add(des)

init testing binary
graph loaded
/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/nginx-{openssl-1.0.1d}{zlib-1.2.11}_bn.dot
edges loaded


In [96]:
nodes = set()
for key in testbin.funcNameFilted.keys():
    if testbin.funcNameFilted[key] != -1:
        nodes.add(key)
print len(nodes-connectednodes)

450


In [101]:
from redis import Redis
from db import db
def getHashMap(configname, storage_object, dim=64):
    db_instance = db()
    db_instance.loadHashMap(configname, storage_object, dim)

    return db_instance
redis_object3 = Redis(host='localhost', port=6379, db=1)
global hashMap3
hashMap3 = getHashMap(["filted1-1gram-binary0", "filted1-1gram-binary1", "filted1-1gram-binary2"], redis_object3, dim=64)
query = []
for node in nodes-connectednodes:
    emb = testbin.funcName2emb[node]
    query.append([emb, node, 1])
knn = hashMap3.querying(query)

using redis
all has done, time: 28.8863739967


In [26]:
dir = '/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx'
#folders = os.listdir(dir)
folders = ['nginx-{openssl-1.0.1d}{zlib-1.2.11}']
for folder in folders:
    knn_all = p.load(open(os.path.join(dir, folder, 'test_kNN_1030_1gram.p')))
    prediction = defaultdict(set)
    cp = []
    wp = []
    fp = []
    trueversion = folder.split('{')[1].split('}')[0]
    trueversion2 = folder.split('{')[2].split('}')[0]
    for [(name, distance), predict, sim] in knn_all:
        if sim > 0.95:
            for [lib, version, func] in predict:
                prediction[name].add((func, sim))
    for func in prediction.keys():
        predict = sorted(prediction[func], key=lambda x: x[1], reverse=True)
        largest = predict[0][1]
        predicts = []
        for i in predict:
            if i[1] == largest:
                predicts.append(i[0])
        #print len(predicts)
        if func in predicts:
            cp.append(func)
            if len(predicts) > 1:
                wp.extend(predicts[:-1])
        elif func in librarynames:
            wp.extend(predicts)
        else:
            fp.extend(predicts)
    print len(cp), len(wp), len(fp), len(cp)*1.0/(len(cp) + len(wp) + len(fp)), len(cp)*1.0/2982

2400 885 131 0.702576112412 0.804828973843


In [3]:
query = []
for node in nodes:
    emb = testbin.funcName2emb[node]
    query.append([emb, node, 1])
knn_all = hashMap3.querying(query)

NameError: name 'nodes' is not defined

In [112]:
len(knn_all)

29343

In [13]:
def runBPversions(queryPath_2gram):
    global LBP_MAX_ITERS
    global THRES
    threshold = THRES
    if type(queryPath_2gram) == type(''):
        query_2gram = p.load(open(queryPath_2gram, 'rb'))
    else:
        query_2gram = queryPath_2gram
    hasDistance = False
    if type(query_2gram[0][0][0]) == type((1,2)):
        hasDistance = True
        
    global_node_dict = defaultdict(list)
    version_set = set()
    g = fg.Graph()
    global_edge_set = set()
    for i in query_2gram:
        if i[2] > threshold:
            if not hasDistance:
                test_src = i[0][0]
                test_des = i[0][1]
            else:
                test_src = i[0][0][0]
                test_des = i[0][0][1]
            if test_src == test_des:
                continue

            funcList = i[1]
            for predicted_func in funcList:
                if hasDistance:
                    querydistance = i[0][1]
                    resultdistance = predicted_func[3]
                    if querydistance != resultdistance:
                        continue

                version = predicted_func[1]
                libraryname = predicted_func[0]
                version_set.add((libraryname, version))
                if (libraryname, version) not in global_node_dict[test_src]:
                    global_node_dict[test_src].append((libraryname, version))

                if (libraryname, version) not in global_node_dict[test_des]:
                    global_node_dict[test_des].append((libraryname, version))
                
                global_edge_set.add((test_src, test_des))
                    
    version_list = list(version_set)
    for key in global_node_dict:
        length = len(version_set)
        priorsBelieve = np.zeros(len(version_set))
        for (libraryname, version) in global_node_dict[key]:
            ind = version_list.index((libraryname, version))
            priorsBelieve[ind] = 1
        rv_func = g.rv(key, length, prior = priorsBelieve)
        
    for (test_src, test_des) in global_edge_set:
        src_rv = g.get_rvs()[test_src]
        des_rv = g.get_rvs()[test_des]
        factor_probability = np.full((len(version_list), len(version_list)), 0.01)
        np.fill_diagonal(factor_probability, 0.99)
        factor1 = g.factor([src_rv, des_rv], potential=factor_probability, name = '')

    # run BP
    #pdb.set_trace()
    iters, converged = g.lbp(global_node_dict, normalize=True, max_iters=LBP_MAX_ITERS, progress=False)
    print 'LBP ran for %d iterations. Converged = %r' % (iters, converged)
    #print
    # Print out the final marginals
    #counts = g.print_rv_marginals(global_dict, normalize=True)
    #counts = count_rv_marginals2(version_list, normalize=True)
    #counts = g.accuracy_of_functions(global_dict, normalize=True)
    #counts = g.print_None_accuracy(global_dict, folder, normalize=True)
    return g, version_list
global LBP_MAX_ITERS
global THRES
LBP_MAX_ITERS =40
THRES = 0.999
knn = p.load(open('/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/test_kNN_1021_2gram.p', 'r'))
g, version_list = runBPversions(knn)

LBP ran for 40 iterations. Converged = 0.9798598949211909


In [14]:
def count_rv_marginals2(g, global_dict, rvs=None, normalize=False):
        # Extract
        from collections import Counter
        tuples = g.rv_marginals(rvs, normalize)
        votes = Counter()
        prediction = dict()
        # Display
        for rv, marg in tuples:
            rv_label = str(rv)
            largest = 0
            for label in marg:
                if label > largest:
                    largest = label
            if type(global_dict)==type([]):
                names = global_dict
            else:
                names = global_dict[rv_label]
                names.append('None')
            for i, label in enumerate(marg):
                if np.isclose(label, largest):
                    if rv_label in prediction:
                        prediction[rv_label].append(names[i])
                    else:
                        prediction[rv_label]=[names[i]]

        keylist = prediction.keys()
        for key in keylist:
            funclist = prediction[key]
            label=set()
            #print key
            for (lib,func) in funclist:
                predicted_label = func.split('{')[0]
                #print '\t', predicted_label
                if predicted_label == 'None':
                    continue
                    # label=set()
                    # break
                label.add(predicted_label)
            votes += Counter(label)

        sorted_count = sorted(votes.items(), key=lambda x: x[1], reverse=True)
        print sorted_count
        return sorted_count
    
count_rv_marginals2(g, version_list)

[('openssl-1.0.1d', 1411), ('openssl-1.0.1f', 92), ('openssl-1.0.1h', 91), ('openssl-1.0.1m', 86), ('openssl-1.0.1c', 79), ('openssl-1.0.1e', 76), ('openssl-1.0.1a', 76), ('openssl-1.0.1b', 76), ('openssl-1.0.1g', 73), ('openssl-1.0.1', 73), ('openssl-1.0.1o', 72), ('openssl-1.0.1n', 72), ('openssl-1.0.0k', 72), ('openssl-1.0.1i', 71), ('openssl-1.0.1p', 70), ('openssl-1.0.1j', 69), ('openssl-1.0.1q', 69), ('openssl-1.0.1r', 69), ('openssl-1.0.1t', 68), ('openssl-1.0.1l', 67), ('openssl-1.0.1k', 67), ('openssl-1.0.1s', 67), ('openssl-1.0.0d', 67), ('openssl-1.0.0b', 65), ('openssl-1.0.0c', 65), ('openssl-1.0.0a', 65), ('openssl-1.0.0g', 64), ('openssl-1.0.0j', 64), ('openssl-1.0.0f', 62), ('openssl-1.0.0e', 62), ('openssl-1.0.0l', 62), ('openssl-1.0.0m', 62), ('openssl-1.0.0h', 62), ('openssl-1.0.0i', 62), ('openssl-1.0.0q', 62), ('openssl-1.0.0n', 61), ('openssl-1.0.0', 61), ('openssl-1.0.0o', 60), ('openssl-1.0.0p', 60), ('openssl-1.0.0t', 59), ('openssl-1.0.0s', 57), ('openssl-1.0.0

[('openssl-1.0.1d', 1411),
 ('openssl-1.0.1f', 92),
 ('openssl-1.0.1h', 91),
 ('openssl-1.0.1m', 86),
 ('openssl-1.0.1c', 79),
 ('openssl-1.0.1e', 76),
 ('openssl-1.0.1a', 76),
 ('openssl-1.0.1b', 76),
 ('openssl-1.0.1g', 73),
 ('openssl-1.0.1', 73),
 ('openssl-1.0.1o', 72),
 ('openssl-1.0.1n', 72),
 ('openssl-1.0.0k', 72),
 ('openssl-1.0.1i', 71),
 ('openssl-1.0.1p', 70),
 ('openssl-1.0.1j', 69),
 ('openssl-1.0.1q', 69),
 ('openssl-1.0.1r', 69),
 ('openssl-1.0.1t', 68),
 ('openssl-1.0.1l', 67),
 ('openssl-1.0.1k', 67),
 ('openssl-1.0.1s', 67),
 ('openssl-1.0.0d', 67),
 ('openssl-1.0.0b', 65),
 ('openssl-1.0.0c', 65),
 ('openssl-1.0.0a', 65),
 ('openssl-1.0.0g', 64),
 ('openssl-1.0.0j', 64),
 ('openssl-1.0.0f', 62),
 ('openssl-1.0.0e', 62),
 ('openssl-1.0.0l', 62),
 ('openssl-1.0.0m', 62),
 ('openssl-1.0.0h', 62),
 ('openssl-1.0.0i', 62),
 ('openssl-1.0.0q', 62),
 ('openssl-1.0.0n', 61),
 ('openssl-1.0.0', 61),
 ('openssl-1.0.0o', 60),
 ('openssl-1.0.0p', 60),
 ('openssl-1.0.0t', 59),


In [11]:
version_list

[('libssl', 'openssl-1.0.0m'),
 ('libcrypto', 'openssl-1.0.2b'),
 ('libssl', 'openssl-0.9.7b'),
 ('libssl', 'openssl-0.9.8x'),
 ('libcrypto', 'openssl-0.9.7h'),
 ('libssl', 'openssl-1.0.0b'),
 ('libssl', 'openssl-0.9.7'),
 ('libz', 'zlib-1.2.7.1'),
 ('libssl', 'openssl-1.0.2o'),
 ('libcrypto', 'openssl-0.9.8u'),
 ('libssl', 'openssl-1.0.1r'),
 ('libssl', 'openssl-1.0.1f'),
 ('libcrypto', 'openssl-1.1.0a'),
 ('libcrypto', 'openssl-1.0.1b'),
 ('libcrypto', 'openssl-1.0.0c'),
 ('libcrypto', 'openssl-1.0.2l'),
 ('libcrypto', 'openssl-1.0.0p'),
 ('libssl', 'openssl-0.9.8n'),
 ('libssl', 'openssl-1.0.0h'),
 ('libcrypto', 'openssl-1.0.2a'),
 ('libssl', 'openssl-1.0.1i'),
 ('libcrypto', 'openssl-0.9.7g'),
 ('libssl', 'openssl-1.0.2b'),
 ('libssl', 'openssl-1.0.0r'),
 ('libcrypto', 'openssl-0.9.8p'),
 ('libssl', 'openssl-1.0.1l'),
 ('libcrypto', 'openssl-1.1.0b'),
 ('libcrypto', 'openssl-1.0.0i'),
 ('libcrypto', 'openssl-1.0.1g'),
 ('libcrypto', 'openssl-0.9.7a'),
 ('libcrypto', 'openssl-1.0.0f

In [ ]:
dir = '/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx'
folders = os.listdir(dir)
folders.sort()
folders.index('')